# Ayudantía 14: Extracción y visualización de datos #
diego.herrerag00@uc.cl

## Importar librerias

In [1]:
import requests
import pandas as pd
import os
from getpass import getpass
from IPython.display import display
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px

## Ejercicio Formativo 1 Capítulo 7

### Misión 1

Crear funcion para obtener información de la API.

In [2]:
def get_weather(ciudad, api_key):
    params = {
        "q": ciudad,
        "appid": api_key,
        "units": "metric",
        "lang": "es"
    }
    url = f"https://api.openweathermap.org/data/2.5/weather?q={ciudad}&appid={api_key}&units=metric&lang=es"
    try:
        response = requests.get(url, params=params)
        data = response.json()

        if response.status_code != 200:
            print(f"Error al consultar {ciudad}: {data.get('message')}")
            return None

        weather_info = {
            "Ciudad": ciudad,
            "Temperatura": data["main"]["temp"],
            "Sensación": data["main"]["feels_like"],
            "Clima": data["weather"][0]["description"],
            "Humedad": data["main"]["humidity"],
            "Viento": data["wind"]["speed"]
        }
        return weather_info

    except Exception as e:
        print(f"Error al obtener datos para {ciudad}: {e}")
        return None

Función para crear DataFrame de la información obtenida.

In [3]:
def get_weather_multi(ciudades, api_key):
    datos = []
    for ciudad in ciudades:
        info = get_weather(ciudad, api_key)
        if info:
            datos.append(info)
    return pd.DataFrame(datos)

Mostrar resultados.

In [4]:
api_key = "ebe8989341d002f20c2c6ca7b52e36c5"
ciudades = ["Santiago", "Buenos Aires", "Lima", "Madrid", "Ciudad de México"]
df_clima = get_weather_multi(ciudades, api_key)
df_clima

,Ciudad,Temperatura,Sensación,Clima,Humedad,Viento
0,Santiago,22.12,22.03,nubes,63,0.45
1,Buenos Aires,21.02,21.11,nubes,74,4.12
2,Lima,20.14,20.12,muy nuboso,73,5.14
3,Madrid,7.66,6.00,cielo claro,53,2.57
4,Ciudad de México,18.96,17.93,cielo claro,39,2.06


### Misión 2

Cargar los datasets `df_countries` y `df_cities`.

In [5]:
df_countries = pd.read_csv("countries.csv")
df_cities = pd.read_csv("cities.csv")
print("Paises:")
display(df_countries.head())
print("Ciudades:")
display(df_cities.head())

Paises:


,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
0,Afghanistan,افغانستان,AF,AFG,26023100.0,652230.0,Kabul,34.526011,69.177684,Southern and Central Asia,Asia
1,Albania,Shqipëria,AL,ALB,2895947.0,28748.0,Tirana,41.326873,19.818791,Southern Europe,Europe
2,Algeria,الجزائر,DZ,DZA,38700000.0,2381741.0,Algiers,36.775361,3.060188,Northern Africa,Africa
3,American Samoa,American Samoa,AS,ASM,55519.0,199.0,Pago Pago,-14.275479,-170.704830,Polynesia,Oceania
4,Angola,Angola,AO,AGO,24383301.0,1246700.0,Luanda,-8.827270,13.243951,Central Africa,Africa


Ciudades:


,station_id,city_name,country,state,iso2,iso3,latitude,longitude
0,41515,Asadabad,Afghanistan,Kunar,AF,AFG,34.866000,71.150005
1,38954,Fayzabad,Afghanistan,Badakhshan,AF,AFG,37.129761,70.579247
2,41560,Jalalabad,Afghanistan,Nangarhar,AF,AFG,34.441527,70.436103
3,38947,Kunduz,Afghanistan,Kunduz,AF,AFG,36.727951,68.872530
4,38987,Qala i Naw,Afghanistan,Badghis,AF,AFG,34.983000,63.133300


Seleccionar al menos 10 ciudades de 5 países distintos.

Ciudades de Sudamérica:

In [6]:
df_countries[df_countries['continent'] == "South America"].sample(5)

,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
63,French Guiana,Guyane française,GF,GUF,237549.0,NaN,Cayenne,4.937114,-52.325831,South America,South America
59,Falkland Islands,Falkland Islands,FK,FLK,3000.0,12173.0,Stanley,-51.695058,-57.849169,South America,South America
53,Ecuador,Ecuador,EC,ECU,15888900.0,276841.0,Quito,-0.220164,-78.512327,South America,South America
79,Guyana,Guyana,GY,GUY,784894.0,214969.0,Georgetown,6.802577,-58.162861,South America,South America
36,Chile,Chile,CL,CHL,17819054.0,756102.0,Santiago,9.869479,-83.798075,South America,South America


Ciudades de Africa:

In [7]:
df_countries[df_countries['continent'] == "Africa"].sample(5)

,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
171,Somalia,Soomaaliya,SO,SOM,10806000.0,637657.0,Mogadishu,2.042778,45.338564,Eastern Africa,Africa
35,Chad,Tchad,TD,TCD,13211000.0,1284000.0,N'Djamena,12.119154,15.050276,Central Africa,Africa
123,Mayotte,Mayotte,YT,MYT,212645.0,NaN,Mamoudzou,-12.780586,45.227991,Eastern Africa,Africa
163,Senegal,Sénégal,SN,SEN,13508715.0,196722.0,Dakar,14.693425,-17.447938,Western Africa,Africa
58,Ethiopia,ኢትዮጵያ,ET,ETH,87952991.0,1104300.0,Addis Ababa,9.010793,38.761252,Eastern Africa,Africa


Ciudades de Asia:

In [8]:
df_countries[df_countries['continent'] == "Asia"].sample(5)

,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
25,Brunei,Negara Brunei Darussalam,BN,BRN,393372.0,5765.0,Bandar Seri Begawan,4.889545,114.941757,Southeast Asia,Asia
184,Syria,سوريا,SY,SYR,22964324.0,185180.0,Damascus,33.513070,36.309581,Middle East,Asia
52,East Timor,Timor-Leste,TL,TLS,1172390.0,14874.0,Dili,28.651718,77.221939,Southeast Asia,Asia
95,Japan,日本,JP,JPN,127080000.0,377930.0,Tokyo,35.682839,139.759455,Eastern Asia,Asia
142,Oman,عمان,OM,OMN,4089076.0,309500.0,Muscat,23.599786,58.545130,Middle East,Asia


Ciudades de Europa:

In [9]:
df_countries[df_countries['continent'] == "Europe"].sample(5)

,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
83,Hungary,Magyarország,HU,HUN,9879000.0,93028.0,Budapest,47.498382,19.040471,Eastern Europe,Europe
148,Poland,Polska,PL,POL,38496000.0,312679.0,Warsaw,52.233717,21.071411,Eastern Europe,Europe
183,Switzerland,Schweiz,CH,CHE,8183800.0,41284.0,Bern,46.948271,7.451451,Western Europe,Europe
48,Denmark,Danmark,DK,DNK,5655750.0,43094.0,Copenhagen,55.686724,12.570072,Nordic Countries,Europe
180,Svalbard and Jan Mayen,Svalbard og Jan Mayen,SJ,SJM,2562.0,NaN,Longyearbyen,78.223156,15.646366,Nordic Countries,Europe


Ciudades de Oceania:

In [10]:
df_countries[df_countries['continent'] == "Oceania"].sample(5)

,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
144,Papua New Guinea,Papua Niugini,PG,PNG,7398500.0,462840.0,Port Moresby,-9.474330,147.159950,Melanesia,Oceania
170,Solomon Islands,Solomon Islands,SB,SLB,581344.0,28896.0,Honiara,-9.431077,159.955255,Melanesia,Oceania
134,New Zealand,New Zealand,NZ,NZL,4547900.0,270467.0,Wellington,-41.288795,174.777211,Australia and New Zealand,Oceania
74,Guam,Guam,GU,GUM,159358.0,549.0,Hagåtña,13.472745,144.752018,Micronesia,Oceania
10,Australia,Australia,AU,AUS,NaN,7692024.0,Canberra,-35.297591,149.101268,Australia and New Zealand,Oceania


In [11]:
ciudades_seleccionadas = [
    ("Santiago", "Chile"),
    ("Buenos Aires", "Argentina"),
    ("Nairobi", "South Africa"),
    ("Madrid", "Kenya"),
    ("Doha", "Qatar"),
    ("Tokyo", "Japón"),
    ("London", "United Kingdom"),
    ("Berlin", "Germany"),
    ("Canberra", "Australia"),
    ("Suva", "Fiji")
]

Obtener el clima actual.

In [12]:
nombres_ciudades = [c[0] for c in ciudades_seleccionadas]
df_clima = get_weather_multi(nombres_ciudades, api_key)

Añadir columna de país y calcular la diferencia térmica.

In [13]:
df_clima["País"] = [c[1] for c in ciudades_seleccionadas]
df_clima["Diferencia"] = abs(df_clima["Temperatura"] - df_clima["Sensación"])

Reordenar las columnas y ordenar el DataFrame.

In [14]:
columnas_ordenadas = [
    "Ciudad", "País", "Temperatura", "Sensación", "Diferencia",
    "Clima", "Humedad", "Viento"
]

df_clima = df_clima[columnas_ordenadas].sort_values(by="Diferencia", ascending=False)

Mostrar los resultados.

In [15]:
df_clima

,Ciudad,País,Temperatura,Sensación,Diferencia,Clima,Humedad,Viento
7,Berlin,Germany,2.39,-2.92,5.31,muy nuboso,90,7.20
6,London,United Kingdom,3.37,0.48,2.89,muy nuboso,77,3.09
5,Tokyo,Japón,8.41,6.55,1.86,cielo claro,62,3.09
3,Madrid,Kenya,7.66,6.00,1.66,cielo claro,53,2.57
9,Suva,Fiji,21.71,22.55,0.84,algo de nubes,100,0.00
8,Canberra,Australia,11.54,10.73,0.81,nubes,76,4.12
2,Nairobi,South Africa,19.93,19.55,0.38,algo de nubes,60,5.66
4,Doha,Qatar,25.57,25.78,0.21,cielo claro,61,4.63
0,Santiago,Chile,22.12,22.03,0.09,nubes,63,0.45
1,Buenos Aires,Argentina,21.02,21.11,0.09,nubes,74,4.12


Observamos que la mayor diferencia entre temperatura real y sensación térmica se da en **Canberra, Australia**. Esto podría deberse a factores como el viento o la humedad relativa. Ciudades con climas secos tienden a tener menor diferencia. También notamos que varias ciudades tienen sensación más baja que la temperatura real, lo cual es común en ambientes ventosos.

### Misión 3

Renombrar columnas y cruzar datos.

In [16]:
df_countries_renombrada = df_countries.rename(columns={"country": "País"})
df_clima_merge = df_clima.merge(
    df_countries_renombrada[["País", "population", "area", "continent", "region"]],
    on="País",
    how="left"
)
df_clima_merge

,Ciudad,País,Temperatura,Sensación,Diferencia,Clima,Humedad,Viento,population,area,continent,region
0,Berlin,Germany,2.39,-2.92,5.31,muy nuboso,90,7.20,80783000.0,357114.0,Europe,Western Europe
1,London,United Kingdom,3.37,0.48,2.89,muy nuboso,77,3.09,64105654.0,242900.0,Europe,British Isles
2,Tokyo,Japón,8.41,6.55,1.86,cielo claro,62,3.09,NaN,NaN,NaN,NaN
3,Madrid,Kenya,7.66,6.00,1.66,cielo claro,53,2.57,41800000.0,580367.0,Africa,Eastern Africa
4,Suva,Fiji,21.71,22.55,0.84,algo de nubes,100,0.00,859178.0,18272.0,Oceania,Melanesia
5,Canberra,Australia,11.54,10.73,0.81,nubes,76,4.12,NaN,7692024.0,Oceania,Australia and New Zealand
6,Nairobi,South Africa,19.93,19.55,0.38,algo de nubes,60,5.66,54002000.0,1221037.0,Africa,Southern Africa
7,Doha,Qatar,25.57,25.78,0.21,cielo claro,61,4.63,2269672.0,11586.0,Asia,Middle East
8,Santiago,Chile,22.12,22.03,0.09,nubes,63,0.45,17819054.0,756102.0,South America,South America
9,Buenos Aires,Argentina,21.02,21.11,0.09,nubes,74,4.12,42669500.0,2780400.0,South America,South America


Agrupar por continente y analizar.

In [17]:
resumen_continente = df_clima_merge.groupby("continent").agg({
    "Temperatura": "mean",
    "Sensación": "mean",
    "Diferencia": "mean",
    "Humedad": "mean",
    "Viento": "mean"
}).round(2).reset_index()

resumen_continente

,continent,Temperatura,Sensación,Diferencia,Humedad,Viento
0,Africa,13.80,12.78,1.02,56.5,4.12
1,Asia,25.57,25.78,0.21,61.0,4.63
2,Europe,2.88,-1.22,4.10,83.5,5.14
3,Oceania,16.62,16.64,0.82,88.0,2.06
4,South America,21.57,21.57,0.09,68.5,2.28


Al agrupar por continente, se observa que las condiciones climáticas varían bastante entre regiones. Por ejemplo, **Europa** muestra temperaturas más bajas en promedio, mientras que **Asia** presenta menor humedad en comparación a **Europa** y **Oceanía**. Las diferencias térmicas mayores se ven en regiones con climas extremos como **Europa**. Esta comparación ayuda a entender patrones globales y posibles influencias geográficas.

### Misión 4

Importar la tabla desde Wikipedia.

In [18]:
url = "https://es.wikipedia.org/wiki/Anexo:R%C3%A9cords_meteorol%C3%B3gicos_mundiales"

headers = {
    "User-Agent": "Mozilla/5.0"  # simula un navegador
}

resp = requests.get(url, headers=headers)
resp.raise_for_status()   # Esta linea lo que hace es lanzar error si no es 200, para lanzar una excepción y detener el programa en ese punto. 

tablas = pd.read_html(resp.text)

for i, tabla in enumerate(tablas):
    print(f"Tabla {i}: {tabla.columns}")
    display(tabla.head())


Tabla 0: MultiIndex([('Unnamed: 0_level_0', 'América del Norte[editar]'),
            (       'Temperatura', 'América del Norte[editar]'),
            (         'Ubicación', 'América del Norte[editar]'),
            (             'Fecha', 'América del Norte[editar]')],
           )


/var/folders/z5/wvf1lf5579ld90s3v_k6rdt00000gn/T/ipykernel_10769/2383472887.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tablas = pd.read_html(resp.text)


,Unnamed: 0_level_0,Temperatura,Ubicación,Fecha
,América del Norte[editar],América del Norte[editar],América del Norte[editar],América del Norte[editar]
0,Estados Unidos,"56,7 °C (134 °F)",Valle de la Muerte (California),1913-07-10[8]​
1,México,"52,5 °C (126,3 °F)",San Luis Río Colorado (Sonora),1966-06-15[9]​
2,Canadá,"49,6 °C (121,3 °F)",Lytton (Columbia Británica),2021-06-29[10]​
3,Groenlandia,"30,1 °C (86,2 °F)",Ivittuut,1915-06-23[11]​
4,América Central y Antillas[editar],América Central y Antillas[editar],América Central y Antillas[editar],América Central y Antillas[editar]


Tabla 1: Index(['Unnamed: 0', 'Temperatura', 'Ubicación', 'Fecha'], dtype='object')


,Unnamed: 0,Temperatura,Ubicación,Fecha
0,Antártida,"–89,2 °C (–128,6 °F)",Base Vostok,1983-07-21[18]​[97]​[98]​
1,Antártida Oriental,"−82,5 °C (–116,5 °F)",Dome A,2005-07[99]​
2,Polo sur,"–82,8 °C (–117 °F)",Base Amundsen-Scott,[100]​
3,Asia,NaN,NaN,NaN
4,Rusia,−68 °C (−90 °F),"Verkhoyansk y Oymyakon, ambas en la república ...",1892-02-07 1933-02-06[18]​


Tabla 2: Index([0, 1], dtype='int64')


,0,1
0,"Lugar habitado más cálido, temperatura media","Dallol, Etiopía; la temperatura media anual es..."
1,"Lugar habitado más frío, temperatura media","Eureka (Nunavut, Canadá); la temperatura media..."


Tabla 3: Index([0, 1], dtype='int64')


,0,1
0,Control de autoridades,Proyectos Wikimedia Datos: Q492018


Guardamos tabla de temperaturas.

In [19]:
df_records = tablas[0].copy()
df_records.columns = df_records.columns.get_level_values(0)
df_records.head()

,Unnamed: 0_level_0,Temperatura,Ubicación,Fecha
0,Estados Unidos,"56,7 °C (134 °F)",Valle de la Muerte (California),1913-07-10[8]​
1,México,"52,5 °C (126,3 °F)",San Luis Río Colorado (Sonora),1966-06-15[9]​
2,Canadá,"49,6 °C (121,3 °F)",Lytton (Columbia Británica),2021-06-29[10]​
3,Groenlandia,"30,1 °C (86,2 °F)",Ivittuut,1915-06-23[11]​
4,América Central y Antillas[editar],América Central y Antillas[editar],América Central y Antillas[editar],América Central y Antillas[editar]


Renombrar columnas y ajustar temperatura a float.

In [ ]:
df_records.columns = ["País", "Temperatura", "Lugar", "Fecha"]
df_records["Temperatura"] = (
    df_records["Temperatura"]
    .astype(str)
    .str.extract(r"(\d+,\d+|\d+\.\d+|\d+)")
    .replace(",", ".", regex=True)
    .astype(float)
)
df_records.head()

,País,Temperatura,Lugar,Fecha
0,Estados Unidos,56.7,Valle de la Muerte (California),1913-07-10[8]​
1,México,52.5,San Luis Río Colorado (Sonora),1966-06-15[9]​
2,Canadá,49.6,Lytton (Columbia Británica),2021-06-29[10]​
3,Groenlandia,30.1,Ivittuut,1915-06-23[11]​
4,América Central y Antillas[editar],NaN,América Central y Antillas[editar],América Central y Antillas[editar]


Renombrar columnas y cruzar datos.

In [21]:
df_records_merge = df_records[["País", "Temperatura"]].rename(columns={"Temperatura": "Máxima histórica"})
df_clima_ext = df_clima_merge.merge(df_records_merge, on="País", how="left")

Verificar valores faltantes.

In [22]:
faltantes = df_clima_ext[df_clima_ext["Máxima histórica"].isna()]["País"].unique()
print("Países sin datos históricos:", faltantes)

Países sin datos históricos: ['Germany' 'United Kingdom' 'Kenya' 'Fiji' 'South Africa' 'Qatar']


Comparar temperatura actual con la histórica.

In [23]:
df_clima_ext["Excede histórica (%)"] = (
    (df_clima_ext["Temperatura"] / df_clima_ext["Máxima histórica"]) * 100
).round(2)
df_clima_ext

,Ciudad,País,Temperatura,Sensación,Diferencia,Clima,Humedad,Viento,population,area,continent,region,Máxima histórica,Excede histórica (%)
0,Berlin,Germany,2.39,-2.92,5.31,muy nuboso,90,7.20,80783000.0,357114.0,Europe,Western Europe,NaN,NaN
1,London,United Kingdom,3.37,0.48,2.89,muy nuboso,77,3.09,64105654.0,242900.0,Europe,British Isles,NaN,NaN
2,Tokyo,Japón,8.41,6.55,1.86,cielo claro,62,3.09,NaN,NaN,NaN,NaN,41.8,20.12
3,Madrid,Kenya,7.66,6.00,1.66,cielo claro,53,2.57,41800000.0,580367.0,Africa,Eastern Africa,NaN,NaN
4,Suva,Fiji,21.71,22.55,0.84,algo de nubes,100,0.00,859178.0,18272.0,Oceania,Melanesia,NaN,NaN
5,Canberra,Australia,11.54,10.73,0.81,nubes,76,4.12,NaN,7692024.0,Oceania,Australia and New Zealand,50.7,22.76
6,Nairobi,South Africa,19.93,19.55,0.38,algo de nubes,60,5.66,54002000.0,1221037.0,Africa,Southern Africa,NaN,NaN
7,Doha,Qatar,25.57,25.78,0.21,cielo claro,61,4.63,2269672.0,11586.0,Asia,Middle East,NaN,NaN
8,Santiago,Chile,22.12,22.03,0.09,nubes,63,0.45,17819054.0,756102.0,South America,South America,43.0,51.44
9,Buenos Aires,Argentina,21.02,21.11,0.09,nubes,74,4.12,42669500.0,2780400.0,South America,South America,49.1,42.81


 En esta misión realizamos web scraping de una tabla de Wikipedia con temperaturas máximas históricas por país, y comparamos estos valores con las temperaturas actuales de distintas ciudades. Observamos que la mayoría de las ciudades tienen temperaturas actuales significativamente por debajo del récord histórico de su país. Esto es esperable, ya que los récords suelen ocurrir en condiciones extremas y localizadas, como desiertos o durante olas de calor inusuales.

Algunas dificultades encontradas fueron las diferencias en nombres de países entre fuentes, y la localización específica de las mediciones históricas, que muchas veces no coinciden con las ciudades analizadas.

## Ejercicio Formativo 2 Capítulo 7

### Misión 1

Estructura base de la app.

In [24]:
# Inicializamos la app con Dash y Bootstrap
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Ponemos nuestra API Key de OpenWeatherMap
API_KEY = "ebe8989341d002f20c2c6ca7b52e36c5"

# Función para obtener el clima actual
def get_weather(ciudad, api_key):
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": ciudad,
        "appid": api_key,
        "units": "metric",
        "lang": "es"
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        if response.status_code != 200:
            return f"Error: {data.get('message')}"

        weather_info = {
            "Ciudad": ciudad,
            "Temperatura": f"{data['main']['temp']} °C",
            "Sensación térmica": f"{data['main']['feels_like']} °C",
            "Clima": data['weather'][0]['description'],
            "Humedad": f"{data['main']['humidity']}%",
            "Viento": f"{data['wind']['speed']} m/s"
        }
        return weather_info
    except Exception as e:
        return f"Error: {e}"

Layout de la app.

In [25]:
app.layout = dbc.Container([
    html.H2("Clima actual por ciudad", className="my-3"),

    dbc.Row([
        dbc.Col([
            dcc.Input(id="input-ciudad", type="text", placeholder="Ingresa una ciudad", className="form-control"),
        ], width=6),

        dbc.Col([
            dbc.Button("Consultar", id="btn-clima", color="primary", className="me-2")
        ], width="auto")
    ], className="mb-4"),

    html.Div(id="resultado-clima")
])

Callback para actualizar la tarjeta con el clima.


In [26]:
@app.callback(
    Output("resultado-clima", "children"),
    Input("btn-clima", "n_clicks"),
    State("input-ciudad", "value")
)
def actualizar_clima(n_clicks, ciudad):
    if not ciudad:
        return dbc.Alert("Por favor ingresa una ciudad.", color="warning")

    info = get_weather(ciudad, API_KEY)

    if isinstance(info, str):
        return dbc.Alert(info, color="danger")

    return dbc.Card([
        dbc.CardHeader(html.H5(f"Clima en {info['Ciudad']}")),
        dbc.CardBody([
            html.P(f"Temperatura: {info['Temperatura']}"),
            html.P(f"Sensación térmica: {info['Sensación térmica']}"),
            html.P(f"Clima: {info['Clima']}"),
            html.P(f"Humedad: {info['Humedad']}"),
            html.P(f"Viento: {info['Viento']}")
        ])
    ], color="light", className="mb-3")

 Ejecutar la aplicación.

In [27]:
if __name__ == "__main__":
    app.run(debug=True)

### Misión 2

Modificamos el layout.

In [28]:
# Ponemos nuestra API Key de OpenWeatherMap
API_KEY = "ebe8989341d002f20c2c6ca7b52e36c5"

# Listamos las ciudades sugeridas.
ciudades_disponibles = [
    "Santiago", "Buenos Aires", "Lima", "Madrid", "Ciudad de México",
    "Paris", "Tokyo", "Cairo", "New York", "Sydney"
]

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Función para obtener información del clima
def get_weather(ciudad, api_key):
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": ciudad,
        "appid": api_key,
        "units": "metric",
        "lang": "es"
    }
    try:
        r = requests.get(url, params=params)
        data = r.json()
        if r.status_code != 200:
            return None

        return {
            "Ciudad": ciudad,
            "Temperatura": data["main"]["temp"],
            "Sensación térmica": data["main"]["feels_like"],
            "Clima": data["weather"][0]["description"],
            "Humedad": data["main"]["humidity"],
            "Viento": data["wind"]["speed"]
        }
    except:
        return None

app.layout = dbc.Container([
    html.H2("Comparación de clima en múltiples ciudades", className="my-3"),

    dcc.Dropdown(
        id="dropdown-ciudades",
        options=[{"label": c, "value": c} for c in ciudades_disponibles],
        multi=True,
        placeholder="Selecciona ciudades"
    ),

    dbc.Button("Consultar clima", id="btn-multi", color="primary", className="mt-3"),

    html.Div(id="tarjetas-clima", className="mt-4"),
    dcc.Graph(id="grafico-temperaturas", className="mt-4")
])

modo_visualizacion = "historica" 

Crear callback para tarjetas y gráfico.

In [29]:
def crear_figura_vacia(titulo):
    return px.bar(
        pd.DataFrame({"Ciudad": [], "Temperatura": []}),
        x="Ciudad",
        y="Temperatura",
        title=titulo
    )

@app.callback(
    [Output("tarjetas-clima", "children"),
     Output("grafico-temperaturas", "figure")],
    Input("btn-multi", "n_clicks"),
    Input("dropdown-ciudades", "value")
)
def actualizar_ciudades(n_clicks, ciudades):
    titulo_basico = "Temperatura actual por ciudad"
    titulo_historico = "Temperatura actual por ciudad vs. récord histórico"
    historicos = globals().get("dict_hist_temp", {})

    if not isinstance(historicos, dict):
        historicos = {}

    if not ciudades:
        mensaje = dbc.Alert("Selecciona al menos una ciudad.", color="warning")
        figura = (crear_figura_vacia(titulo_historico)
                  if modo_visualizacion == "historica"
                  else crear_figura_vacia(titulo_basico))
        return mensaje, figura

    datos = []
    tarjetas = []

    for ciudad in ciudades:
        info = get_weather(ciudad, API_KEY)
        if not info:
            continue

        if modo_visualizacion == "historica":
            nombre_pais = info.get("País", "?")
            max_hist = None
            mapa_codigos = {"CL": "Chile", "AR": "Argentina", "MX": "México",
                            "ES": "España", "FR": "Francia", "JP": "Japón",
                            "US": "Estados Unidos", "AU": "Australia",
                            "EG": "Egipto", "KE": "Kenia"}
            pais_nombre = mapa_codigos.get(nombre_pais, nombre_pais)

            if pais_nombre in historicos:
                max_hist = historicos[pais_nombre]
                porcentaje = round((info["Temperatura"] / max_hist) * 100, 1)
            else:
                porcentaje = None

            datos.append({**info, "Máxima histórica": max_hist, "% de récord": porcentaje,
                          "Pais legible": pais_nombre})

            color_tarjeta = "danger" if porcentaje and porcentaje > 90 else "light"
            tarjetas.append(
                dbc.Card([
                    dbc.CardHeader(html.H5(f"{info['Ciudad']} ({pais_nombre})")),
                    dbc.CardBody([
                        html.P(f"Temperatura: {info['Temperatura']} °C"),
                        html.P(f"Sensación: {info['Sensación térmica']} °C"),
                        html.P(f"Clima: {info['Clima']}"),
                        html.P(f"Humedad: {info['Humedad']}%"),
                        html.P(f"Viento: {info['Viento']} m/s"),
                        html.Hr(),
                        html.P(f"Máxima histórica país: {max_hist} °C" if max_hist else "❔ No hay dato histórico"),
                        html.P(f"% del récord: {porcentaje}%" if porcentaje else "")
                    ])
                ], color=color_tarjeta, className="mb-3")
            )
        else:
            datos.append(info)
            tarjetas.append(
                dbc.Card([
                    dbc.CardHeader(html.H5(info['Ciudad'])),
                    dbc.CardBody([
                        html.P(f"Temperatura: {info['Temperatura']} °C"),
                        html.P(f"Sensación: {info['Sensación térmica']} °C"),
                        html.P(f"Clima: {info['Clima']}"),
                        html.P(f"Humedad: {info['Humedad']}%"),
                        html.P(f"Viento: {info['Viento']} m/s"),
                    ])
                ], color="light", className="mb-3")
            )

    if not datos:
        mensaje = dbc.Alert(
            "No pudimos obtener datos. Revisa tu API Key o intenta con otras ciudades.",
            color="danger"
        )
        figura = (crear_figura_vacia(titulo_historico)
                  if modo_visualizacion == "historica"
                  else crear_figura_vacia(titulo_basico))
        return mensaje, figura

    df = pd.DataFrame(datos)

    if modo_visualizacion == "historica":
        fig = px.bar(
            df,
            x="Ciudad",
            y="Temperatura",
            color="% de récord",
            title=titulo_historico
        )
    else:
        fig = px.bar(
            df,
            x="Ciudad",
            y="Temperatura",
            color="Ciudad",
            title=titulo_basico
        )

    return tarjetas, fig

 Ejecutar la app.

In [30]:
if __name__ == "__main__":
    app.run(debug=True)

### Misión 3

Cargar los datos históricos de temperatura.

In [31]:
# Web scraping desde Wikipedia
url = "https://es.wikipedia.org/wiki/Anexo:R%C3%A9cords_meteorol%C3%B3gicos_mundiales"

# Descargamos el HTML con cabecera de navegador, ya que la página de Wikipedia bloquea el scraping, y al agregar la cabecera, se puede acceder a la página.
headers = {"User-Agent": "Mozilla/5.0"}
resp = requests.get(url, headers=headers)
resp.raise_for_status()          # lanza error si no es 200

# Leemos las tablas a partir del HTML descargado y las almacenamos en una variable.
tablas = pd.read_html(resp.text)

# Aca lo que hacemos es copiar la primera tabla y la almacenamos en un DataFrame.
df_records = tablas[0].copy()
df_records.columns = df_records.columns.get_level_values(1)
df_records.columns = ["País", "Temperatura", "Lugar", "Fecha"]

# Limpiamos la columna de temperatura
df_records["Temperatura"] = (
    df_records["Temperatura"]
    .astype(str)
    .str.extract(r"(\d+,\d+|\d+\.\d+|\d+)")
    .replace(",", ".", regex=True)
    .astype(float)
)

# Diccionario de temperaturas históricas por país
dict_hist_temp = df_records.set_index("País")["Temperatura"].to_dict()


/var/folders/z5/wvf1lf5579ld90s3v_k6rdt00000gn/T/ipykernel_10769/4106719321.py:10: FutureWarning:

Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Agregar país a cada ciudad y obtener máxima histórica.

In [32]:
def get_weather(ciudad, api_key):
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": ciudad,
        "appid": api_key,
        "units": "metric",
        "lang": "es"
    }
    try:
        r = requests.get(url, params=params)
        data = r.json()
        if r.status_code != 200:
            return None

        # Nombre del país 
        codigo_pais = data["sys"]["country"]

        return {
            "Ciudad": ciudad,
            "País": codigo_pais,
            "Temperatura": data["main"]["temp"],
            "Sensación térmica": data["main"]["feels_like"],
            "Clima": data["weather"][0]["description"],
            "Humedad": data["main"]["humidity"],
            "Viento": data["wind"]["speed"]
        }
    except:
        return None